In [2]:
from lxml import etree
from lxml import html
from collections import defaultdict
from pprint import pprint
import json

In [58]:
# Parse the TOAD generated file
doc = html.parse("table.html")

In [59]:
## Global structures ################################################################################################
# Global list of all the tables. Each table is a dict (icc_table) were key is the table name and value is a dict with 
# 4 keys : columns, foreign_keys, keys and indexes and values are list of keys with definitions 
icc = []

In [60]:
## Routines ##########################################################################################################
def populate_table(table):
    # icc_table structure to be returned
    icc_table = {}
    # extract table name
    table_name = table.xpath('div[@class="caption1"]/p/text()').pop()
    ############################################################
    # Identify and set number of sections for table.
    ############################################################
    # Some tables have 4 sections (columns, fkeys, keys, indexes) as offsets of table[@class="tabformat"]. Second option
    # is 3 sections so columns, keys, indexes. Whern there are 2 sections, columns is 0 and indexes 1 and when there is
    # only one columns is columns
    # So i will check and set a global flag first
    ############################################################
    number_of_sections = len(table.xpath('table[@class="tabformat"]'))
    ############################################################
    # process columns
    ############################################################
    # List will be "value" of columns dict for icc_table
    columns_result = []
    # Obtain all column information for given table
    # columns is always 0 independent on number of sections 
    columns = table.xpath('table[@class="tabformat"]')[0]
    # call process_columns to populate columns_result
    columns_result = process_columns(columns)
    ############################################################
    # process foreign keys , keys and indexes
    ############################################################
    fkeys_result = []
    keys_result = []
    indexes_result = []
    # Process depende on number of sections for the table
    # print "table name : {}, number of sections : {}".format(table_name, number_of_sections)
    
    if number_of_sections == 4:
        foreign_keys = table.xpath('table[@class="tabformat"]')[1]
        fkeys_result = process_fkeys(foreign_keys)        
        keys = table.xpath('table[@class="tabformat"]')[2]
        keys_result = process_keys(keys)
        indexes = table.xpath('table[@class="tabformat"]')[3]
        indexes_result = process_indexes(indexes)
    elif number_of_sections == 3:
        keys = table.xpath('table[@class="tabformat"]')[1]
        keys_result = process_keys(keys)
        indexes = table.xpath('table[@class="tabformat"]')[2]
        indexes_result = process_indexes(indexes)
    elif number_of_sections == 2:
        indexes = table.xpath('table[@class="tabformat"]')[1]
        indexes_result = process_indexes(indexes)
        
    
    icc_table[table_name] = dict(columns=columns_result, fkeys=fkeys_result, keys=keys_result, indexes=indexes_result)
    return icc_table
    
def process_columns(columns):
    # list to process individual columns
    column = []
    # list to append individual dicts with columns definitions
    columns_result = []
    # get first all rows for columns
    col = columns.xpath('./tr')
    # for each row , check if none empty and process text 
    for coli in col:
        if coli.xpath('./td[@class="tabdata"]/text()') != []:
            column.append(coli.xpath('./td[@class="tabdata"]/text()'))
    # Now, process text of each row to extract column info (name, datatype, not_null and key)
    for col in column:
        dict_columns = {}
        if len(col) == 3:
            dict_columns['column_name'] = col[0]
            dict_columns['datatype']    = col[1]
            dict_columns['not_null']    = col[2]
        else:
            dict_columns['key']         = col[0]
            dict_columns['column_name'] = col[1]
            dict_columns['datatype']    = col[2]
            dict_columns['not_null']    = col[3]
    # Add built dict to columns list of column definition
        columns_result.append(dict_columns)
    # return list with dicts for each column definition
    return columns_result
 
def process_fkeys(foreign_keys):
    # list to process individual f_keys
    f_key = []
    # list to append individual dicts with f_keys definitions
    f_keys_result = []
    # get first all rows for f_keys
    fks = foreign_keys.xpath('./tr')
    # for each row , check if none empty and process text 
    for fk in fks:
        if fk.xpath('./td[@class="tabdata"]/text()') != []:
            # print fk.xpath('./td[@class="tabdata"]/text()')
            tabdata_struct = []
            for f in fk:
                if f.text != None:
                    tabdata_struct.append(f.text)
                else:
                    tabdata_struct.append('No value')
            f_key.append(tabdata_struct)
    # Now, process text of each row to extract f_key info (name, datatype, child and parent)
    for fk in f_key:
        ## print fk
        dict_fore_keys = {} 
        if len(fk) == 4:
            dict_fore_keys['fore_key_name'] = fk[0]
            dict_fore_keys['fore_key_type'] = fk[1]
            dict_fore_keys['parent_table']  = fk[2]
            dict_fore_keys['child_table']   = fk[3] 
        else:
            dict_fore_keys['fore_key_name'] = fk[0]
            dict_fore_keys['fore_key_type'] = fk[1]
        # Add built dict to f keys list of f kets definition
        f_keys_result.append(dict_fore_keys)
    # return list with dicts for each f keys definition
    return f_keys_result
            
def process_keys(keys):
    # list to process individual keys
    ind_key = []
    # list to append individual dicts with keys definitions
    keys_result = []
    # get first all rows for columns
    col = keys.xpath('./tr')
    # for each row , check if none empty and process text 
    for coli in col:
        if coli.xpath('./td[@class="tabdata"]/text()') != []:
            ind_key.append(coli.xpath('./td[@class="tabdata"]/text()'))
    # Now, process text of each row to extract keys info (type, constraint, column)
    for ke in ind_key:
        dict_keys = {}
        if len(ke) == 2:
            dict_keys['key_type']    = ke[0]
            dict_keys['constraint_name'] = ke[1]
        else:
            dict_keys['key_type']        = ke[0]
            dict_keys['constraint_name'] = ke[1]
            dict_keys['columns']         = ke[2]
    # Add built dict to keys list of keys definition
        keys_result.append(dict_keys)
    # return list with dicts for each column definition
    return keys_result
        
def process_indexes(indexes):
    # list to process individual indexes
    idx = []
    # list to append individual dicts with indexes definitions
    indexes_result = []
    # get first all rows for indexes
    col = indexes.xpath('./tr')
    # for each row , check if none empty and process text 
    for coli in col:
        if coli.xpath('./td[@class="tabdata"]/text()') != []:
            idx.append(coli.xpath('./td[@class="tabdata"]/text()'))
    # Now, process text of each row to extract index info (name, columns)
    for ix in idx:
        dict_indexes = {}
        if len(ix) == 2:
            dict_indexes['index_name'] = ix[0]
            dict_indexes['columns']    = ix[1]
        else:
            dict_indexes['index_name'] = ix[0]
    # Add built dict to indexes list of indexes definition
        indexes_result.append(dict_indexes)
    # return list with dicts for each column definition
    return indexes_result

In [61]:
# Look for "item" class which contain table information
all_tables = doc.xpath('//*[@class="item"]')
# call populate_table for each table in all_tables to build the icc_table structure. populate_table will return an 
# icc_table structure to append to icc
for table in all_tables:
    icc.append(populate_table(table))

In [65]:
# icc

In [66]:
json_str = json.dumps(icc)

In [67]:
with open('data.json', 'w') as f:
    json.dump(json_str, f)

In [69]:
with open('data.json', 'r') as r:
    data = json.load(r)

In [71]:
resp = json.loads(data.decode('utf-8'))

In [72]:
pprint(resp)

[{u'BUSINESSDATA_COLOMBIA.ACCSTAT': {u'columns': [{u'column_name': u'EXTERNAL_REFERENCE',
                                                   u'datatype': u'VARCHAR2 (64 Char)',
                                                   u'not_null': u'False'},
                                                  {u'column_name': u'DESCRIPTION',
                                                   u'datatype': u'VARCHAR2 (20 Char)',
                                                   u'not_null': u'True'},
                                                  {u'column_name': u'LONGDESCR',
                                                   u'datatype': u'VARCHAR2 (420 Char)',
                                                   u'not_null': u'False'},
                                                  {u'column_name': u'ACTIVE',
                                                   u'datatype': u'NUMBER (1)',
                                                   u'not_null': u'True'},
                             